<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Ingeniería de features

Primer modelo, métricas y visualización del dataset de propiedades\
v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
if os.access('propiedades.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/propiedades.csv > propiedades.csv
    else:
        !wget propiedades.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/propiedades.csv

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
df = pd.read_csv("propiedades.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
df.head()

In [ ]:
# Sacamos todas las filas de la tabla las cuales el campo "m2" o "ambientes" se encuentre vacio
df_clean = df.copy()
df_clean.dropna(subset=['m2'], inplace=True)
df_clean.dropna(subset=['ambientes'], inplace=True)

# Otra manera de obtener el mismo resultado:
#df_clean = df[df['m2'].notna()]
#df_clean = df_clean[df_clean['ambientes'].notna()]

In [ ]:
# ¿Cuántos datos quedaron para analizar?
# (filas, columnas)
df_clean.shape

## Fin de la limpieza
Se finalizó la limpieza sacando aquellas filas que no eran de interes, se puede observar que la cantidad de filas hasta el momento es menor a la cantidad de filas con la que se comnezó el análisis

In [ ]:
print('Cantidad de datos en observacion:', df_clean.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
df_clean.head()

Se puede observar que hay alquileres en dolares, solo nos quedaremos con aquellos alquileres en pesos

In [ ]:
# Filtramos el dataframe, solos nos quedamos con los alquileres en pesos
df_ars = df_clean[df_clean['moneda'] == 'ARS']
print('Cantidad de datos en observacion:', df_ars.shape[0])

In [ ]:
# Renderizamos el gráfico utilizando el diagrama de cajas
#df_ars.boxplot(column=['m2'])
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot()
sns.boxplot(x=df_ars['m2'], ax=ax)
ax.grid('dashed')

Se puede observar que hay muchos alquileres "outliers", es decir que se escapan de la tendencia. Filtraremos aquellos que no aportan al análisis

In [ ]:
# Nos quedamos solamente con los deparamentos menor a 200m2 y mayor a 50m2
# Utilizamos el concepto de "máscara" tal cual fue visto en Numpy
# para filtrar el dataframe de propiedads deseado
propiedades = df_ars[(df_ars['m2'] < 200) & (df_ars['m2'] > 50)]

In [ ]:
print('Cantidad de datos en observacion:', propiedades.shape[0])

In [ ]:
# Graficar la tendencia
sns.scatterplot(x=propiedades['m2'], y=propiedades['precio'], color='b', label='precio vs m2')
plt.show()

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

El primer paso es obtener los datos que serán la entrada del sistema (X) y los datos que serán la salida del modelo estimador (y)

In [ ]:
X = propiedades[['m2']].values
#X = propiedades[['ambientes','m2']].values
y = propiedades['precio'].values

Siguiente paso es dividir el dataset en entrenamiento (train) y evaluación (test). Utilizaremos el criterio 70%30%

In [ ]:
from sklearn.model_selection import train_test_split
# Fijamos un "random_state" constante para que siempre el dataset se parta de la misma forma
# para poder repetir los ensayos
# Ojo! Los dataset de train y test son array numpy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Ahora generaremos un modelo "base", el cual podremos utilizarlo para evaluar en la próxima instancia contra el modelo que construyamos con inteligencia artificial

In [ ]:
# Creamos el modelo base
class MeanBaseModel():
    def __init__(self):
        self.W = 0

    def fit(self, X, y):
        self.W = np.mean(y / X)

    def predict(self, X):
        return X * self.W

In [ ]:
# Entrenar nuestro modelo base (un promediador)
mean_model = MeanBaseModel()
mean_model.fit(X_train, y_train)
y_hat_base = mean_model.predict(X_test)
print(f"Precio del m2 promedio: ${mean_model.W:.2f}")

In [ ]:
# Sino hubieramos utilizado el modelo generado, las líneas de código serían:
precio_m2 = y_train / X_train
promedio_precio_m2 = precio_m2.mean()
y_hat_base = X_test * promedio_precio_m2
print(f"Precio del m2 promedio: ${promedio_precio_m2:.2f}")

In [ ]:
sns.scatterplot(x=X_test[:,0], y=y_hat_base[:,0], color='darkGreen', label='y_hat_model')
#plt.scatter(X_test, y_hat_base, color='darkGreen', label='y_hat_model')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
propiedades.plot.scatter(['m2'], ['precio'], color='b', ax=ax)
ax.scatter(X_test, y_hat_base, color='darkGreen', label="base model")
ax.legend()
plt.show()

Como se observa en el último gráfico, la línea verde aproxima bastante a la distribución de puntos azules, pero debería "apuntar" un poco más arriba. En el próximo encuentro discutiremos porque sucede este fenomeno

In [ ]:
# Calcular los errores del modelo base
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

print('Error promedio   MAE - Modelo Base:', mean_absolute_error(y_test, y_hat_base))
print('Error cuadrático MSE - Modelo Base:', mean_squared_error(y_test, y_hat_base))

## Conclusión
Sin utilizar inteligencia artificial, unicamente con nuestro conocimientos sobre los datos y estadística hemos generado un primer modelo base competirá contra los algoritmos de inteligencia artificial que veremos en la siguiente etapa